In [134]:
from src.utils.myOptimization import solveGroupProblem
from src.utils.myPrediction import generate_random_features, customPredictionModel
import numpy as np
import cvxpy as cp
import torch
import torch.nn as nn
import pandas as pd
from src.utils.features import get_all_features

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings
# Suppress warnings
warnings.filterwarnings("ignore")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [135]:
df = pd.read_csv('E:\\User\\Stevens\\MyRepo\\Organized-FDFL\\src\\data\\data.csv')
df = df.sample(n=50,random_state=1)

# Normalized cost 
cost = np.array(df['cost_t_capped']).reshape(-1, 1) * 100
# Ensure cost is at least 0.001
cost = np.maximum(cost, 1)  # Ensure cost is at least 0.001


# All features, standardized
features = df[get_all_features(df)].values
scaler = StandardScaler()
features = scaler.fit_transform(features)

# True benefit, predictor label 
true_benefit = np.array(df['benefit'].values).reshape(-1, 1) *100
true_benefit = np.maximum(true_benefit, 1)


# Group labels, 0 is White (Majority), 1 is Black
race = np.array(df['race'].values).reshape(-1, 1)


In [136]:
true_benefit

array([[ 1.        ],
       [35.00343584],
       [ 1.        ],
       [ 1.        ],
       [ 5.88235294],
       [30.87915071],
       [32.04230367],
       [ 2.94117647],
       [ 1.        ],
       [ 1.        ],
       [ 5.88235294],
       [ 1.        ],
       [ 2.94117647],
       [ 1.        ],
       [ 5.88235294],
       [20.19598257],
       [46.60914432],
       [50.62378517],
       [ 1.        ],
       [ 1.        ],
       [ 1.        ],
       [ 2.94117647],
       [23.24236875],
       [ 1.        ],
       [ 2.94117647],
       [30.29132306],
       [51.1538104 ],
       [ 1.        ],
       [ 1.        ],
       [ 1.        ],
       [20.19598257],
       [ 1.        ],
       [ 5.88235294],
       [23.52941176],
       [ 1.        ],
       [ 2.94117647],
       [43.47694379],
       [ 1.        ],
       [ 8.82352941],
       [ 2.94117647],
       [ 1.        ],
       [ 1.        ],
       [36.98288317],
       [ 2.94117647],
       [ 2.94117647],
       [ 5

In [137]:
def alpha_fairness_group_utilities(benefit, allocation, group, alpha):
    """
    Compute group-wise alpha-fairness utilities.
    """
    groups = np.unique(group)
    utils = []
    for k in groups:
        mask = (group == k)
        Gk = float(mask.sum())
        # Compute average utility in group k
        util_k = (benefit[mask] * allocation[mask]).sum(axis=0).mean()  # mean total utility per individual in group
        if alpha == 1:
            val = np.log(util_k) if util_k > 0 else -np.inf
        elif alpha == 0:
            val = util_k
        elif alpha == float('inf'):
            # Min utility as min total utility)
            val = (benefit[mask] * allocation[mask]).sum(axis=0).min()
        else:
            val = util_k**(1 - alpha) / (1 - alpha)
        utils.append(val)
    return np.array(utils).sum()

In [138]:
class FairRiskPredictor(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            # Output layer
            nn.Linear(64, 1),
            nn.Softplus()
        )
            
    def forward(self, x):
        return self.model(x)

In [139]:
def AlphaFairness(util, alpha):
    if isinstance(util, torch.Tensor):
        util = util.detach().cpu().numpy() if isinstance(util, torch.Tensor) else util
    if alpha == 1:
        return np.sum(np.log(util))
    elif alpha == 0:
        return np.sum(util)
    elif alpha == 'inf':
        return np.min(util)
    else:
        return np.sum(util**(1-alpha) / (1-alpha))

def solveIndProblem(benefit, cost, alpha, Q):


    d = cp.Variable(benefit.shape, nonneg=True)

    utils = cp.multiply(benefit, d)
    constraints = [d >= 0, cp.sum(cp.multiply(cost, d)) <= Q]

    if alpha == 'inf':
        t = cp.Variable()
        objective = cp.Maximize(t)
        constraints.append(utils >= t)
    elif alpha == 1:
        objective = cp.Maximize(cp.sum(cp.log(utils)))
    elif alpha == 0:
        objective = cp.Maximize(cp.sum(utils))
    else:
        objective = cp.Maximize(cp.sum(utils**(1-alpha)) / (1-alpha))

    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=True, warm_start=True, mosek_params={'MSK_IPAR_LOG': 1})

    if problem.status != 'optimal':
        print(f"Warning: Problem status is {problem.status}")

    optimal_decision = d.value
    optimal_value = AlphaFairness(d.value * benefit, alpha)

    return optimal_decision, optimal_value

In [140]:
sol, obj = solveIndProblem(true_benefit, cost, alpha=1, Q=20)

                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Jun 19 02:38:07 PM: Your problem has 50 variables, 51 constraints, and 0 parameters.
(CVXPY) Jun 19 02:38:07 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 19 02:38:07 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 19 02:38:07 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 19 02:38:07 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 19 02:38:07 PM: Compiling problem (target solver=MOSEK).
(C

In [141]:
sol,obj

(array([[0.40000294],
        [0.02494761],
        [0.40000294],
        [0.1849047 ],
        [0.09525544],
        [0.04365808],
        [0.02054524],
        [0.14968703],
        [0.40000294],
        [0.19646099],
        [0.40000113],
        [0.40000294],
        [0.04762791],
        [0.00789734],
        [0.40000113],
        [0.053278  ],
        [0.01023918],
        [0.00399997],
        [0.03613306],
        [0.40000294],
        [0.40000294],
        [0.04306104],
        [0.0296549 ],
        [0.26194362],
        [0.00982386],
        [0.03742137],
        [0.03022467],
        [0.40000294],
        [0.40000294],
        [0.40000294],
        [0.03572062],
        [0.19646099],
        [0.07666899],
        [0.00399949],
        [0.26194362],
        [0.39999822],
        [0.00913785],
        [0.12573779],
        [0.06415161],
        [0.06985275],
        [0.13097706],
        [0.17463315],
        [0.00464313],
        [0.1309758 ],
        [0.39999821],
        [0